In [16]:
%matplotlib inline

import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Define User Interface Function

In [17]:
# Create a user interface to set parameters for both heaters

def PCR_thermal_cycle(filename):

    filename_text = widgets.Text(value=filename, description="Filename")

    heat_rate = widgets.FloatSlider(value = 0.2, min=0.1, max=0.35, step=0.01, description='Heat Rate')
    cool_rate = widgets.FloatSlider(value = 0.2, min=0.1, max=0.35, step=0.01, description='Cooling Rate')
    cycles = widgets.IntSlider(value=3, min=1, max=30, description="PCR Cycles")

    int_denaturation_label1 = widgets.Label('Initial Denaturation', layout=widgets.Layout(width='100 px'))
    int_denaturation_label2 = widgets.Label('degrees C', layout=widgets.Layout(width='80px'))
    int_denaturation_label3 = widgets.Label('seconds', layout=widgets.Layout(width='80px'))
    int_denaturation_temp = widgets.FloatSlider(value=94, min=92, max=98, step=0.1, description='Temperature', readout_format='0.1f')
    int_denaturation_dur = widgets.BoundedIntText(value=90, min=60, max=120, step=1, description='Duration')
    int_denaturation = widgets.HBox([int_denaturation_label1, int_denaturation_temp,  int_denaturation_label2 , int_denaturation_dur, int_denaturation_label3])
    
    denaturation_label1 = widgets.Label('Denaturation', layout=widgets.Layout(width='80 px'))
    denaturation_label2 = widgets.Label('degrees C', layout=widgets.Layout(width='80px'))
    denaturation_label3 = widgets.Label('seconds', layout=widgets.Layout(width='80px'))
    denaturation_temp = widgets.FloatSlider(value=94, min=92, max=98, step=0.1, description='Temperature', readout_format='0.1f')
    denaturation_dur = widgets.BoundedIntText(value=60, min=30, max=120, step=1, description='Duration')
    denaturation = widgets.HBox([denaturation_label1, denaturation_temp,  denaturation_label2 , denaturation_dur, denaturation_label3])

    annealing_label1 = widgets.Label('Annealing', layout=widgets.Layout(width='80 px'))
    annealing_label2 = widgets.Label('degrees C', layout=widgets.Layout(width='80px'))
    annealing_label3 = widgets.Label('seconds', layout=widgets.Layout(width='80px'))
    annealing_temp = widgets.FloatSlider(value=60, min=50, max=65, step=0.1, description='Temperature', readout_format='0.1f')
    annealing_dur = widgets.BoundedIntText(value=60, min=30, max=120, step=1, description='Duration')
    annealing = widgets.HBox([annealing_label1, annealing_temp,annealing_label2, annealing_dur, annealing_label3])

    extension_label1 = widgets.Label('Extension', layout=widgets.Layout(width='80 px'))
    extension_label2= widgets.Label('degrees C', layout=widgets.Layout(width='80px'))
    extension_label3 = widgets.Label('seconds', layout=widgets.Layout(width='80px'))
    extension_temp = widgets.FloatSlider(value=72, min=68, max=72, step=0.1, description='Temperature', readout_format='0.1f')
    extension_dur = widgets.BoundedIntText(value=90, min=30, max=180, step=1, description='Duration')
    extension = widgets.HBox([extension_label1, extension_temp, extension_label2, extension_dur, extension_label3])
    
    fin_extension_label1 = widgets.Label('Final Extension', layout=widgets.Layout(width='120 px'))
    fin_extension_label2= widgets.Label('degrees C', layout=widgets.Layout(width='80px'))
    fin_extension_label3 = widgets.Label('seconds', layout=widgets.Layout(width='80px'))
    fin_extension_temp = widgets.FloatSlider(value=72, min=68, max=72, step=0.1, description='Temperature', readout_format='0.1f')
    fin_extension_dur = widgets.BoundedIntText(value=90, min=30, max=300, step=1, description='Duration')
    fin_extension = widgets.HBox([fin_extension_label1, fin_extension_temp, fin_extension_label2, fin_extension_dur, fin_extension_label3])

    def create_setpoint_trajectory(
            filename_text,
            heat_rate, cool_rate, cycles,
            int_denaturation_temp, int_denaturation_dur,
            denaturation_temp, denaturation_dur, 
            annealing_temp, annealing_dur,
            extension_temp, extension_dur,
            fin_extension_temp, fin_extension_dur):
        Ta = 22
        tp = [0]
        Tp = [Ta]   

        for k in range(0, cycles):
            
            #execute initial denaturation
            if k == 0:
                # initial denaturation
                tp.append(tp[-1] + (int_denaturation_temp - Tp[-1]) / heat_rate)
                Tp.append(int_denaturation_temp)
                tp.append(tp[-1] + int_denaturation_dur)
                Tp.append(int_denaturation_temp)
                
                
                # annealing
                tp.append(tp[-1] - (annealing_temp - denaturation_temp) / cool_rate)
                Tp.append(annealing_temp)
                tp.append(tp[-1] + annealing_dur)
                Tp.append(annealing_temp)

                # extension
                tp.append(tp[-1] + (extension_temp - annealing_temp) / heat_rate)
                Tp.append(extension_temp)
                tp.append(tp[-1] + extension_dur)
                Tp.append(extension_temp)
            
            #execute final extension
            elif k == cycles-1:
                # denaturation
                tp.append(tp[-1] + (denaturation_temp - Tp[-1]) / heat_rate)
                Tp.append(denaturation_temp)
                tp.append(tp[-1] + denaturation_dur)
                Tp.append(denaturation_temp)

                # annealing
                tp.append(tp[-1] - (annealing_temp - denaturation_temp) / cool_rate)
                Tp.append(annealing_temp)
                tp.append(tp[-1] + annealing_dur)
                Tp.append(annealing_temp)

                # final extension
                tp.append(tp[-1] + (fin_extension_temp - annealing_temp) / heat_rate)
                Tp.append(fin_extension_temp)
                tp.append(tp[-1] + fin_extension_dur)
                Tp.append(fin_extension_temp)
            
            #complete a normal cycle
            else:
                # denaturation
                tp.append(tp[-1] + (denaturation_temp - Tp[-1]) / heat_rate)
                Tp.append(denaturation_temp)
                tp.append(tp[-1] + denaturation_dur)
                Tp.append(denaturation_temp)

                # annealing
                tp.append(tp[-1] - (annealing_temp - denaturation_temp) / cool_rate)
                Tp.append(annealing_temp)
                tp.append(tp[-1] + annealing_dur)
                Tp.append(annealing_temp)

                # extension
                tp.append(tp[-1] + (extension_temp - annealing_temp) / heat_rate)
                Tp.append(extension_temp)
                tp.append(tp[-1] + extension_dur)
                Tp.append(extension_temp)
            
            

        tp.append(tp[-1] - (Ta - Tp[-1]) / cool_rate)
        Tp.append(Ta)
        fig, ax = plt.subplots(1, 1, figsize=(1 + max(tp)/500, 5))
        ax.plot(tp, Tp)
        ax.set_xlabel('time [sec] ')
        ax.set_ylabel('temperature [deg C]')
        ax.set_title('PCR Temperature Program')
        ax.plot(tp, Tp, 'r.', ms=3)
        ax.grid(True)
        df = pd.DataFrame()
        df['tp'] = tp
        df['Tp'] = Tp
        df.to_csv(filename_text)

    display(filename_text)
    display(heat_rate)
    display(cool_rate)
    display(cycles)
    display(int_denaturation)
    display(denaturation)
    display(annealing)
    display(extension)
    display(fin_extension)

    out = widgets.interactive_output(create_setpoint_trajectory, {
        'filename_text': filename_text,
        'heat_rate': heat_rate, 
        'cool_rate': cool_rate,
        'cycles': cycles,
        'int_denaturation_temp' : int_denaturation_temp,
        'int_denaturation_dur' : int_denaturation_dur,
        'denaturation_temp': denaturation_temp,
        'denaturation_dur': denaturation_dur,
        'annealing_temp': annealing_temp,
        'annealing_dur': annealing_dur,
        'extension_temp': extension_temp,
        'extension_dur': extension_dur,
        'fin_extension_temp': fin_extension_temp,
        'fin_extension_dur': fin_extension_dur,
        })

    display(out)

# Input PCR Thermal Protocols and Store in a .csv File

In [18]:
print("Please enter a file name with a .csv extension")
userinput= input('Enter a file name:')
filename = 'pcr/'+ userinput


PCR_thermal_cycle(filename)

Please enter a file name with a .csv extension
Enter a file name:test.csv


Text(value='pcr/test.csv', description='Filename')

FloatSlider(value=0.2, description='Heat Rate', max=0.35, min=0.1, step=0.01)

FloatSlider(value=0.2, description='Cooling Rate', max=0.35, min=0.1, step=0.01)

IntSlider(value=3, description='PCR Cycles', max=30, min=1)

Output()